In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('/content/Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report


X=df[['Age','Fare','Sex','Embarked']]
y=df['Survived']

numeric_features=['Age','Fare']
categorical_features=['Sex','Embarked']

numeric_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

categorical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


full_pipeline.fit(X_train,y_train)

y_pred=full_pipeline.predict(X_test)

print('Accuracy',accuracy_score(y_test,y_pred))
print('Confusion_matrix',confusion_matrix(y_test,y_pred))
print('Classification_report',classification_report(y_test,y_pred))

Accuracy 0.7262569832402235
Confusion_matrix [[82 23]
 [26 48]]
Classification_report               precision    recall  f1-score   support

           0       0.76      0.78      0.77       105
           1       0.68      0.65      0.66        74

    accuracy                           0.73       179
   macro avg       0.72      0.71      0.72       179
weighted avg       0.72      0.73      0.73       179



In [8]:
from sklearn.model_selection import GridSearchCV
params={
    'classifier__max_depth':[3,5,7,10,15],
    'classifier__min_samples_split':[2,5,10,20],
    'classifier__min_samples_leaf':[1,2,5,10]

}

# Corrected: Passed full_pipeline object and changed 'CV' to 'cv'
grid=GridSearchCV(full_pipeline, params, cv=5, scoring='accuracy')

grid.fit(X_train,y_train)

print('Best_accuracy',grid.best_score_)
print('Best_parameters',grid.best_params_)

best_model=grid.best_estimator_

y_pred=best_model.predict(X_test)

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print('final Accuracy',accuracy_score(y_test,y_pred))
print('final Confusion_matrix',confusion_matrix(y_test,y_pred))
print('final Classification_report',classification_report(y_test,y_pred))

Best_accuracy 0.7921304048064611
Best_parameters {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 5, 'classifier__min_samples_split': 20}
final Accuracy 0.7262569832402235
final Confusion_matrix [[90 15]
 [34 40]]
final Classification_report               precision    recall  f1-score   support

           0       0.73      0.86      0.79       105
           1       0.73      0.54      0.62        74

    accuracy                           0.73       179
   macro avg       0.73      0.70      0.70       179
weighted avg       0.73      0.73      0.72       179

